# Exploratory Data Analysis (EDA) for Food Reviews Dataset

## Datasets Overview


In [ ]:
# Import required libraries
import pandas as pd
from numpy.ma.extras import row_stack

In [73]:
# Load the data from the CSV and TXT files
bad_lines = []
def collect_bad_line(line):
    bad_lines.append(line)
    return None  # skip

nutrition_df = pd.read_csv('data/Nutrition.csv', sep=";")
restaurants_df = pd.read_csv('data/Restaurants.csv', sep=";")
reviews_df = pd.read_csv('data/Reviews.txt', sep="\t")
recipes_df = pd.read_csv("data/Recipes.csv", sep=";", engine="python", on_bad_lines=collect_bad_line)

print("loaded:", recipes_df.shape)
print("bad lines captured:", len(bad_lines))
print("example bad line:", bad_lines[0][:2000] if bad_lines else "none")


loaded: (402037, 13)
bad lines captured: 0
example bad line: none


In [71]:
print('====== Nutrition Dataset ======')
print('Number of rows: ', nutrition_df.shape[0])
print('Number of columns: ', nutrition_df.shape[1])
print('Number of Unique Recipe Categories: ', len(list(set(nutrition_df['RecipeCategory'].unique()))))
print('Number of Null Values: ', nutrition_df.isnull().sum().sum())
print('The rows of the null values: ', [r for r in nutrition_df[nutrition_df.isnull().any(axis=1)].index])
print('\n')
nutrition_df.head()

====== Nutrition Dataset ======
Number of rows: 81411
Number of columns: 12
Number of Unique Values in 'RecipeCategory': 242
Number of Null Values: 21
The rows of the null values: [1369, 3416, 3526, 3645, 4590, 4591, 75670, 76570, 80674, 80701, 80790, 81410]


,Name,Description,RecipeCategory,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,Frozen Desserts,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2
1,Biryani,Make and share this Biryani recipe from Food.com.,Chicken Breast,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4
2,Best Lemonade,This is from one of my first Good House Keepi...,Beverages,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3
3,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,Soy/Tofu,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3
4,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,Vegetable,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3


In [64]:
print('====== Restaurants Dataset ======')
print('Number of rows: ', restaurants_df.shape[0])
print('Number of columns: ', restaurants_df.shape[1])
print('Number of Unique Countries: ', len(list(set(restaurants_df['Country'].unique()))))
print('Number of Null Values: ', restaurants_df.isnull().sum().sum())
print('The rows of the null values: ', [r for r in restaurants_df[restaurants_df.isnull().any(axis=1)].index])
print('\n')
restaurants_df.head()

====== Restaurants Dataset ======
Number of rows:  9550
Number of columns:  21
Number of Unique Countries:  15
Number of Null Values:  0
The rows of the null values:  []




,Restaurant ID,Restaurant Name,Country,City,Address,Locality Verbose,Longitude,Latitude,Cuisines,Currency,...,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes,Nummber of dishes in cuisines,Average cost of two in USD
0,1600219,12212,India,Nashik,"Shop 10, Ramrajya Building 7, Samarth Nagar, N...","College Road, Nashik",73.754636,20.006690,Fast Food,Indian Rupees(Rs.),...,0,0,0,2,3.5,3.70,3,80,1,4.664
1,17057397,'Ohana,USA,Orlando,"1600 Seven Seas Drive, Lake Buena Vista, FL 32830","Disney World Area, Orlando",-81.585226,28.405437,Hawaiian,Dollar($),...,0,0,0,3,4.5,4.65,5,1151,1,45.000
2,18222559,{Niche} - Cafe & Bar,India,New Delhi,"2nd & 3rd Floor, M-16, M Block, Outer Circle, ...","Connaught Place, New Delhi",77.222507,28.631516,"North Indian, Chinese, Italian, Continental",Indian Rupees(Rs.),...,0,0,0,3,4.1,4.25,4,492,4,17.490
3,113702,@Mango,India,Ahmedabad,"Opposite Sindhu Bhawan, Bodakdev, Ahmedabad","Bodakdev, Ahmedabad",72.501764,23.040163,"North Indian, Continental, Mexican, Italian",Indian Rupees(Rs.),...,0,0,0,3,4.1,4.25,4,769,4,9.328
4,3100446,#45,India,Mangalore,"Ground Floor, Trinity Commercial Complex, Near...","Attavar, Mangalore",0.000000,0.000000,Cafe,Indian Rupees(Rs.),...,0,0,0,2,3.6,3.70,3,209,1,6.996


In [77]:
print('====== Reviews Dataset ======')
print('Number of rows: ', reviews_df.shape[0])
print('Number of columns: ', reviews_df.shape[1])
print('Number of Unique Authors: ', len(list(set(reviews_df['AuthorId'].unique()))))
print('Number of Null Values: ', reviews_df.isnull().sum().sum())
print('The rows of the null values: ', len(reviews_df[reviews_df.isnull().any(axis=1)].index))
print('\n')
reviews_df.head()

====== Reviews Dataset ======
Number of rows:  317099
Number of columns:  7
Number of Unique Authors:  87644
Number of Null Values:  373388
The rows of the null values:  105379




,ReviewId,RecipeId,AuthorId,AuthorName,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,9,4523,2046,Gay Gilmore ckpt,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
2,14,44,2085,Tony Small,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
3,21,148,2156,Darlene Blythe,Would someone please check the Nutrition Facts...,2000-06-02T10:01:00Z,2000-06-02T10:01:00Z
4,22,517,2046,Gay Gilmore ckpt,thought this was terrific!,2000-02-25T09:02:00Z,2000-02-25T09:02:00Z


In [78]:
print('====== Reviews Dataset ======')
print('Number of rows: ', recipes_df.shape[0])
print('Number of columns: ', recipes_df.shape[1])
print('Number of Unique Recipe Categories: ', len(list(set(recipes_df['RecipeCategory'].unique()))))
print('Number of Null Values: ', recipes_df.isnull().sum().sum())
print('The rows of the null values: ', len(recipes_df[recipes_df.isnull().any(axis=1)].index))
print('\n')
recipes_df.head()


====== Reviews Dataset ======
Number of rows:  402037
Number of columns:  13
Number of Unique Recipe Categories:  295
Number of Null Values:  500854
The rows of the null values:  360568




,RecipeId,Name,CookTime,PrepTime,DatePublished,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,PT24H,PT45M,1999-08-09T21:46:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,PT25M,PT4H,1999-08-29T13:12:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,PT5M,PT30M,1999-09-05T19:52:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,PT20M,PT24H,1999-09-03T14:54:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,PT30M,PT20M,1999-09-19T06:19:00Z,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.0,NaN,"c(""Mix everything together and bring to a boil..."


## Fixing Datasets
Dealing with bad lines, duplicates, and null values